### Notebook to process data about drug use

Developed by John Richmond and Helen Richmond

The notebook used fuzzy logic and matching to group pain medications into sets and then consolidate to using a unique name for each drug.  This is necessary to ensure that drugs that have been mis-spelt are not classified differently to the drug with the correct spelling.

After consolidation of the drug names analysis of different usage patterns is carried out

In [ ]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#### Create a list of common brand names.  
This will be used to search for drugs named by brand so that they can be replaced by the correct name in the final datafrsame

In [ ]:
brand_names = ["Nu-Seals 75", "Lyflex", "Bupeaze", "Butec", "BuTranz", "Transtec", "Axsain", "Zacin", 
    "Zapian", "Solpadeine", "Codipar", "Kapake", "Tylex", "Volterol", "Arthrotec", "Arcoxia", "Fencino patches",
    "Neurontin", "Fenbid gel", "Ibugel", "Ibuleve gel", "Nurofen", "Powergel",  "Sevredol", "MST Continus", 
    "Morphgesic", "Zomorph", "Oramorph", "Naprosyn", "Stirlescent", "Oxyact", "Longtec", "OxyContin", 
    "Oxylan", "Lynlor", "OxyNorm", "Shortec", "Losec", "Mandanol", "Calpol", "Panadol", "Feldene",
    "Alzain", "Lyrica", "Palexia", "Ationdo", "Marol", "Tradorec", "Zydol"]

#### Create a list of pain med drug names to be used for this work

In [ ]:
drug_names = [
    'Amitriptyline', 
    'Aspirin',
    'Baclofen',
    'Buprenorphine',
    'Capsaicin',
    'Celecoxib',
    'Co-codamol',
    'Co-dydramol',
    'Codeine',
    'Diazepam',
    'Diclofenac Sodium',
    'Dihydrocodiene',
    'Duloxetine',
    'Etoricoxib',
    'Fentanyl',
    'Gabapentin',
    'Ibuprofen',
    'Ketoprofen',
    'Meloxicam',
    'Morphine Sulphate',
    'Naproxen',
    'Oxycodone',
    'Paracetamol',
    'Piroxicam',
    'Pregabalin',
    'Tapentadol',
    'Tramadol'
]

#### Create a list of the drugs that are controlled (must be a subset of the above drug_names 

In [ ]:
controlled_drugs = [
 'Buprenorphine',
 'Diazepam',
 'Fentanyl',
 'Gabapentin',
 'Morphine Sulphate',
 'Oxycodone',
 'Pregabalin',
 'Tapentadol',
 'Tramadol']

### Read in the base datafile and a datafile with additional variables
Note that both files are linked by having common ssid columns

Assume that the datafiles are in the data folder of the repository.  If not change the path_to_data variable below to the location of the datafiles

In [ ]:
path_to_data = Path(os.getcwd()).parent / "data"

Load the main data into a Pandas DataFrame

In [ ]:
file_name = "BOOST_RawData3.csv"
full_path = os.path.join(path_to_data, file_name)
initial_df = pd.read_csv(full_path)

Read in the additional data csv file

In [ ]:
additional_data = 'BOOST_additional_variables.csv'
add_df = pd.read_csv(os.path.join(path_to_data, additional_data))

### Remove unwanted columns from the additional DataFrame  

In [ ]:
add_columns_to_retain = [
    'ssid', 'RRAMP_treatment_arm','base_pain_intensity', 'base_ODI', 'fuPQ6_ODI', 'Change_6M_ODI_all'
]
add_df = add_df[add_columns_to_retain]
add_df.head()

### Check details of the main DataFrame

In [ ]:
# Check how many entries there are
len(initial_df) 

In [ ]:
initial_df.loc[5, 'base_medication_name_7']

In [ ]:
initial_df.columns

### Create a list of all of the drug names in the dataframe.
All of the drugs are in the columns starting with 'base_medication_name', hence create a list of these columns to support processing them.

After that a set of unique names will be created.  This will include mis-spelt names and brand names which will then have to be replaced by the correct drug names

In [ ]:
columns = [name for name in initial_df.columns if name.startswith("base_medication_name")]

In [ ]:
columns

In [ ]:
all_names = []
for column in columns:
    names = initial_df[column].values.tolist()
    for name in names:
        all_names.append(name)
unique_names = set(all_names)

In [ ]:
len(all_names), len(unique_names)

In [ ]:
unique_names

Identify all of the brand names in the set of unique names

In [ ]:
for name in unique_names:
    if name in brand_names:
        print(name)

### Create dictionary for each drug
To facilitate replacing all of the mis-spelt or brand names with the correct drug name a dictionary will be created that lists all of the names in the unique_names set with the correct drug_name as the key

This will be done using the fuzzywuzzy library.

Following trial and error it has been found that best results are obtained using the partial ratio method of that library, with a threshold or 75%.  This will require some manual cleaning and manipulation afterwards but works well

In [ ]:
def get_matches(drug_names, unique_names, pr_threshold, tr_threshold):
    unique_names_arr = np.asarray(list(unique_names))
    pr_drugs_dict = {}
    tr_drugs_dict = {}
    for drug in drug_names:
        partial_ratios = []
        token_ratios = []
        for name in unique_names:
            partial_ratio = fuzz.partial_ratio(drug.lower(), name)
            partial_ratios.append(partial_ratio)
            token_ratio = fuzz.token_sort_ratio(drug, name)
            token_ratios.append(token_ratio)
        pr_arr = np.asarray(partial_ratios)
        tr_arr = np.asarray(token_ratios)
        pr_matches = unique_names_arr[np.nonzero(pr_arr>pr_threshold)]
        tr_matches = unique_names_arr[np.nonzero(tr_arr>tr_threshold)]
        pr_drugs_dict[drug] = pr_matches.tolist()
        tr_drugs_dict[drug] = tr_matches.tolist()
    return pr_drugs_dict, tr_drugs_dict

In [ ]:
pr_matches_75, tr_matches_75 = get_matches(drug_names, unique_names, 75, 75)

In [ ]:
tr_matches_75

In [ ]:
pr_matches_75

#### Manually correct the above dictionaries and add in brand names where necessary

In [ ]:
final_dict = pr_matches_75
# Note - removed 'Zapai' from below,
final_dict['Capsaicin'] = ['Capsaicin']

final_dict['Co-codamol'] = [
    'Cocodamol',
    'Co-codamol',
    'Co-Codamol',
    'co codamol effervescent'
]

final_dict['Co-dydramol'] = [
    'Co-Drydramol',
    'CO-dydramol',
    'Co-Dydramol',
    'Co-dydramol',
    'Codydramol'
]

# removed 'dihydrocodiene',   'dihydracodeine',  'dihydrcodeine', from below
final_dict['Codeine'] = [
    'codeine sulphate',
    'codeine phosphate',
    'codiene',
    'codeine phosalate',
    'codeine'
]

# removed ' ' from below, 
final_dict['Diclofenac Sodium'] = ['diclofenac', 'Diclofenac Sodium', 'Diclofenac']

# removed 'Codiene' from below
final_dict['Dihydrocodiene'] = [
    'Dihydrocodiene',
    'Dihydrcodeine',
    'Dihydracodeine',
    'Dihydrocoedrene'
]

# removed 'Fluxetine', 'Fluoxetine' from below
final_dict['Duloxetine'] = ['Duloxetine']

# removed , 'Estradiol', added brands 'Tradorac', 'Marol' from below
final_dict['Tramadol'] = ['Tramadol', 'Tradorac', 'Marol']

# Brands
final_dict['Pregabalin'] =  ['Pregabalin', 'Pregablin', 'Lyrica']
final_dict['Morphine Sulphate'] = ['Morphine Sulphate', 'Butran 10 morphine patches', 'Oramorph']
final_dict['Diclofenac Sodium'] = ['diclofenac', 'Diclofenac Sodium', 'Diclofenac', 'Volterol']
final_dict[ 'Ibuprofen'] = [
    'Ibuprofen',
    'ibuprofen',
    'Ibuprofen pain relief',
    'Ibuprofen Gel',
    'Ibuprofen gel',
    'Fenbid gel'
]
final_dict['Paracetamol'] = [
    'Panadol/Paracetamol',
    'Takes paracetamol from pharmacy',
    'Paracetemol',
    'Paracetamol',
    'Panadol'
]
final_dict['Buprenorphine'] = ['Butec']
final_dict['Fentanyl'] = ['Fentanyl lozenge', 'Fencino patches']
final_dict['Oxycodone'] = ['Oxycodone Hydrochloride', 'Losec']
final_dict

Save the dictionary for subsequent use

In [ ]:
import pickle

In [ ]:
dict_name = 'drug_dict.pkl'
dict_path = path_to_data / Path(dict_name)
with open(dict_path, 'wb') as dict_file:
    pickle.dump(final_dict, dict_file)

At this point the user should move to notebook 2, 

In [ ]:
dict_name = "new_final_dict.pkl"
dict_path = path_to_data / Path(dict_name)
with open(dict_path, 'rb') as dict_file:
    final_dict2 = pickle.load(dict_file)

In [ ]:
final_dict==final_dict2

In [ ]:
for key in final_dict.keys():
    if final_dict[key] != final_dict2[key]:
        print(f'{key}')

In [ ]:
key = 'Aspirin'
print(f"{final_dict[key]}, \n {final_dict2[key]}")

#### Create new array with just the medication columns

In [ ]:
###  Create new dataframe with corrected values
columns_to_use = ["ssid"]+columns
columns_to_use

In [ ]:
new_df = initial_df[columns_to_use]
new_df = new_df.set_index('ssid')

### Loop around replacing mis-spelt names with correct ones

In [ ]:
for drug in drug_names:
    new_df = new_df.replace(final_dict[drug], drug)

In [ ]:
new_df.head(6)

In [ ]:
# Save clean meds dataframe
new_df.to_csv(os.path.join(path_to_dir, 'clean_pain_meds.csv'))

### Merge in the additional data

In [ ]:
# Set inded as ssid to be the same as new_df
add_df = add_df.set_index('ssid')
add_df.head()

In [ ]:
# Using the default merge 'inner' since in this case both dataframes have the same indexed and rows
# If this is not the case it might be necessary to adjust the merge
merged_df = new_df.merge(add_df, left_index=True, right_index=True)

In [ ]:
merged_df

### Process the DataFrame to add additional columsn with counts of the pain meds being taken by each patient

In [ ]:
def count_pain_meds(row: pd.Series, drug_set: list) -> int:
    """ count the number of entries the patient is taking that are in the drug set given
    
    args:
        row (pd.Series): series representing a row of a DataFrame
        drug_set (list): list of the drugs to be included in the count
        
    return:
        count (int): Number of drugs in the set being taken by the patient
    """
    count=0
    for col in row:
        if col in drug_set:
            count +=1
    return count


def get_class_of_drug(row: pd.Series) -> int:
    """ returns a value to represent the class of the strongest pain drug being taken by a patient
    
    args:
        row (pd.Series): series representing a row of a DataFrame
        
    returns:
        int: 0 if no pain meds, 1 if no controlled drugs, 2 if the patient in using controlled drugs
    """
    contr_drugs = row['Num_contr_meds']
    non_cont_drugs = row['Num_pain_meds']
    if non_cont_drugs == 0 & contr_drugs==0:
        return 0
    if contr_drugs > 0:
        return 2
    else:
        return 1

In [ ]:
# Add additional columns defining the number of pain meds total and the number of controlled pain meds
merged_df['Num_pain_meds'] = merged_df.apply(lambda x: count_pain_meds(x, drug_names), axis=1)
merged_df['Num_contr_meds'] = merged_df.apply(lambda x: count_pain_meds(x, controlled_drugs), axis=1)
merged_df.head(20)

In [ ]:
# Add a column to indicate the class of the strongest drug
merged_df['class_drugs'] = merged_df.apply(get_class_of_drug, axis=1)
merged_df.head()

In [ ]:
merged_df

In [ ]:
merged_df['class_drugs'].sum()

In [ ]:
# Save the updated dataframe
merged_df.to_csv(os.path.join(path_to_dir, "merged_df.csv"))

### Group the values to compare changes in base pain levels by drug class

In [ ]:
mean_merged_df = merged_df.groupby('class_drugs').mean()
mean_merged_df

In [ ]:
mean_merged_df.to_csv(os.path.join(path_to_dir, "means_of_merged_df.csv"))

#### Create DataFrame for cases where follow up assessments have been made
We also need to clean the DataFrame columns where a space has been added when no data exists 

In [ ]:
follow_up_df = merged_df.loc[merged_df['Change_6M_ODI_all'] != ' ']

In [ ]:
follow_up_df.to_csv(os.path.join(path_to_dir, "follow_up_df.csv"))

In [ ]:
def convert(row):
    return int(row['Change_6M_ODI_all'])

def convert2(row):
    return int(row['fuPQ6_ODI'])


In [ ]:
follow_up_df = follow_up_df.copy()

Convert strings to ints to facilitate grouping

In [ ]:
follow_up_df['Change_6M_ODI_all'] = follow_up_df['Change_6M_ODI_all'].map(lambda x: int(x))

In [ ]:
follow_up_df['fuPQ6_ODI'] = follow_up_df['fuPQ6_ODI'].map(lambda x: int(x))

In [ ]:
follow_up_groups_df = follow_up_df.groupby('class_drugs').mean()
follow_up_groups_df

In [ ]:
follow_up_groups_df.to_csv(os.path.join(path_to_dir, "follow_up_groups_df.csv"))

### Generate Aggregate Statistics

In [ ]:
np.sum(new_df.values==(drug_names[0]))

In [ ]:
count_dict = {}
for drug in drug_names:
    count_dict[drug] = np.sum(new_df.values == drug)

In [ ]:
count_dict

In [ ]:
total_drug_counts = pd.DataFrame.from_dict(count_dict, orient='index', columns=['Count'])
total_drug_counts

In [ ]:
total_drug_counts.to_csv(os.path.join(path_to_dir, 'total_drug_counts.csv'))

### Number of people on multiple pain meds

In [ ]:
agg_df = merged_df.reset_index()

In [ ]:
agg_df

In [ ]:
agg_df.loc[:,['ssid', 'Num_pain_meds']].groupby('Num_pain_meds').count()

In [ ]:
cntr_pain_meds_plus_opioids = ['Gabapentin', 'Tramadol', 'Pregabalin', 'Morphine Sulphate', 'Fentanyl', 'Oxycodone', 
                          'Buprenorphine', 'Diazepam', 'Tapentadol', 'Co-codamol', 'Co-dydramol', 'Codeine', 
                          'Dihydrocodiene', 'Co-proxamol']
cntr_pain_meds = ['Gabapentin', 'Tramadol', 'Pregabalin', 'Morphine Sulphate', 'Fentanyl', 'Oxycodone', 
                          'Buprenorphine', 'Diazepam', 'Tapentadol', 'Co-proxamol']

In [ ]:
agg_df['totals'] = 1
agg_df

In [ ]:

agg_df.groupby('Num_pain_meds')['totals'].count()

Alternative way to acheive the above using numpy bincount

In [ ]:
np.bincount(agg_df['Num_pain_meds'])

In [ ]:
drug_use_counts = pd.DataFrame(np.bincount(agg_df['Num_pain_meds']), columns=['Total'])
drug_use_counts = drug_use_counts.rename_axis('Number_of_drugs')
drug_use_counts

In [ ]:
# Get all rows with 3 drugs and pain intensity > 1
agg_df.query("(Num_pain_meds >= 3) & (base_pain_intensity > 1)")

### Explore ways to query the dataframe 

In [ ]:
# Get all rows with 3 drugs and that use Tremadol
agg_df.query("(Num_pain_meds >= 3) & (base_medication_name_1 == 'Tramadol')")

In [ ]:
# Get all rows with 3 drugs and that use Tremadol
agg_df.query("(Num_pain_meds >= 3) & ('Tramadol' in base_medication_name_1)")

In [ ]:
drug = 'Tramadol'
query_string = ""
for column_name in columns[0:4]:
    query_string = query_string + f"('{drug}' in {column_name}) or "
query_string = query_string[:-4]
query_string

In [ ]:
full_string = "(Num_pain_meds >= 3) & (" + query_string + ")"
full_string

In [ ]:
agg_df.query(full_string)

In [ ]:
# Use numpy == to generate a binary array and then any to find user columns
drug_cols = agg_df[columns].values
takes_drug = np.any(drug == drug_cols, axis=1)
takes_drug, len(takes_drug), drug_cols.shape

In [ ]:
# use the same approach to filter the rows of the dataframe directly
agg_df[np.any(agg_df[columns].values == drug, axis=1)]

In [ ]:
columns[0]

In [ ]:
new_df2 = new_df.copy()
new_df2 = new_df2.reset_index()
new_df2.head(10)

In [ ]:
row_set = new_df2.iloc[0].values
row_set

In [ ]:
base_names

In [ ]:
count = 0
for col in row_set:
    if col in base_names:
        count +=1
    print(count)

In [ ]:
np.sum(row_set == base_names[1])

In [ ]:
for name in base_name:
    

In [ ]:
def count_pain_meds(row):
    count=0
    for col in row:
        if col in base_names:
            count +=1
    return count

In [ ]:
new_df2['Num_pain_meds'] = new_df2.apply(lambda x: count_pain_meds(x), axis=1)

In [ ]:
new_df2.head(10)

In [ ]:
new_df2.to_csv(os.path.join(path_to_dir, "number_of_pain_meds.csv"))

In [ ]:
new_df2.groupby("Num_pain_meds",axis=0).sum()

### Controlled drugs

In [ ]:
controlled_drugs = [
 'Buprenorphine',
 'Diazepam',
 'Fentanyl',
 'Gabapentin',
 'Morphine Sulphate',
 'Oxycodone',
 'Pregabalin',
 'Tapentadol',
 'Tramadol']

In [ ]:
def count_controlled_meds(row):
    count=0
    for col in row:
        if col in controlled_drugs:
            count +=1
    return count

In [ ]:
new_df2['Num_controlled_meds'] = new_df2.apply(lambda x: count_controlled_meds(x), axis=1)

In [ ]:
new_df2.head(20)

In [ ]:
new_df2.to_csv(os.path.join(path_to_dir, "number_of_pain_meds.csv"))

In [ ]:
new_df2.iloc[3]

In [ ]:
new_df

In [ ]:
final

In [ ]:
base_names[0], final_dict[base_names[0]]

In [ ]:
np.nonzero(initial_df.values == 'Panadol')

In [ ]:
initial_df.iloc[9, 12:29]

In [ ]:
new_df = new_df.replace(final_dict[base_names[0]], base_names[0])

In [ ]:
new_df.iloc[9]

### Re-process agg_df with same methodology as new data

In [ ]:
agg_df = agg_df.iloc[:, 0:-4]
agg_df

In [ ]:
cntr_pain_meds_plus_opioids = ['Gabapentin', 'Tramadol', 'Pregabalin', 'Morphine Sulphate', 'Fentanyl', 'Oxycodone', 
                          'Buprenorphine', 'Diazepam', 'Tapentadol', 'Co-codamol', 'Co-dydramol', 'Codeine', 
                          'Dihydrocodiene', 'Co-proxamol']
cntr_pain_meds = ['Gabapentin', 'Tramadol', 'Pregabalin', 'Morphine Sulphate', 'Fentanyl', 'Oxycodone', 
                          'Buprenorphine', 'Diazepam', 'Tapentadol', 'Co-proxamol']
all_pain_meds = list(final_dict.keys())

In [ ]:
def count_pain_meds(row: pd.Series, drug_set: list) -> int:
    """ count the number of entries the patient is taking that are in the drug set given
    
    args:
        row (pd.Series): series representing a row of a DataFrame
        drug_set (list): list of the drugs to be included in the count
        
    return:
        count (int): Number of drugs in the set being taken by the patient
    """
    count=0
    for col in row:
        if col in drug_set:
            count +=1
    return count


def get_class_of_drug(row: pd.Series, col_to_test) -> int:
    """ returns a value to represent the class of the strongest pain drug being taken by a patient
    
    args:
        row (pd.Series): series representing a row of a DataFrame
        
    returns:
        int: 0 if no pain meds, 1 if no controlled drugs, 2 if the patient in using controlled drugs
    """
    contr_drugs = row[col_to_test]
    non_cont_drugs = row['All_pain_meds']
    if non_cont_drugs == 0 & contr_drugs==0:
        return 0
    if contr_drugs > 0:
        return 2
    else:
        return 1

In [ ]:
summary_dict = {
    'All_pain_meds': all_pain_meds,
    'Num_cont_meds': cntr_pain_meds,
    'Num_cont_op_meds': cntr_pain_meds_plus_opioids
}

In [ ]:
def add_summary_to_df(df, summary_dict):
    for key, value in summary_dict.items():
        df[key] = df.apply(lambda x: count_pain_meds(x, value), axis=1)
    return df

In [ ]:
agg_df = add_summary_to_df(agg_df, summary_dict)

In [ ]:
agg_df.head()

In [ ]:
from functools import partial

In [ ]:
assign_class_controlled = partial(get_class_of_drug, col_to_test="Num_cont_meds")
assign_class_opioids = partial(get_class_of_drug, col_to_test="Num_cont_op_meds")

In [ ]:
agg_df['drug_class'] = agg_df.apply(lambda x: assign_class_controlled(x), axis=1)
agg_df['drug_class_op'] = agg_df.apply(lambda x: assign_class_opioids(x), axis=1)

In [ ]:
agg_df.head(20)

In [ ]:
agg_df.to_csv('updated_aggregate_data.csv')

In [ ]:
drug_names = list(final_dict.keys())
drug_names

In [ ]:
count_dict = {}
for drug in drug_names:
    count_dict[drug] = np.sum(agg_df.values == drug)

In [ ]:
count_df = pd.DataFrame.from_dict(data=count_dict, orient='index', columns=['count'])
count_df

In [ ]:
count_df = count_df.sort_values('count', ascending=False)

In [ ]:
count_df

In [ ]:
count_df.to_csv("counts_from_original.csv")